# 2021 Refinitiv Natural Gas - Analyst Test
*Prepared by Amina Talipova.*
## Below is my approach to solve the basic set of three problems: 
2.	Evaluate natural gas storage; 
 


### I solve this task with the logic described in the "Task 2 explanation.xlsx" supporting document

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import os
from pathlib import Path


In [2]:
#setting the path to find file, here to demonstrate the code I use Henry Hub prices

path = os.getcwd()
filePath = path + '\HenryHubPrompt.csv'

In [3]:
# reading file

df = pd.read_csv(filePath)

In [4]:
# dealing with dates format

df.set_index('Exchange Date',inplace=True)
print(df.head())

               Close
Exchange Date       
30-Sep-20      2.527
29-Sep-20      2.561
28-Sep-20      2.101
25-Sep-20      2.139
24-Sep-20      2.248


In [5]:
# checking out the max-min for information

print(df.Close.argmax())
print(df.Close.argmin())

472
67


In [6]:
# Lets realize the logic (demonstrated in the Excel file "Task 2 explanation") of the trade strategy when we buy at the 
# record low and sell at the record high prices
# Here the output is an array of positions where even positions are the end of decreasing at the lower peak and 
# odd positions are the end of increasing at the higher peak

def findPeaks(arr):
    '''
    findPeaks () returns the array of indexes fo those days when we sell or buy 
    as list of pairs [[buy_day1, sell_day1],...,[[buy_dayN, sell_dayN]]]
    '''
    indexes = [] 
    lower = True
    prev = arr[0]
    for (pos, val) in enumerate(arr):
        if pos == 0:
            continue
        if (lower and val > prev) or (not lower and val < prev):
            lower = not lower
            indexes.append(pos - 1) # previous day is end of sequence
        else:
            prev = val
  # the last element hasn't been added; it is always the end of an increasing or decreasing sequences
    indexes.append(len(arr) - 1)
    return indexes

In [7]:
# Finally, we need to write the function that will take our logic of trading and multiply storage capacity and 
# prices to calculate the maximum profit

def mxProf(arr, capacity):
    '''
    mxProf() returns max_profit and the trade strategy as list of pairs [[buy_day1, sell_day1],...,[[buy_dayN, sell_dayN]]]
    inputs: 
    arr - array of prices
    capacity - storage capacity
    '''
    profits = 0
    if len(arr) < 2: # here is writen the condtion that we cannot sell and buy on one day 
        return profits, []
    seqs = findPeaks(arr)

    if len(seqs) == 1: # if our array is dectreasing in a constant way; or minimize loss 
        prev = arr[1]
        ind = 1
        maxDiff = arr[1] - arr[0]
        for (pos, val) in enumerate(arr):
            if pos == 0 or pos == 1:
                continue
            if val - prev > maxDiff:
                maxDiff = val - prev
                ind = pos
            prev = val
        return [[ind - 1, ind]]
  
    if len(seqs) % 2 == 1: # define when the lowering is ended
        seqs = seqs[:len(seqs)-1]
  
  # here we sell at the highest peak, buy at the the lowest, and wait between
    days = []
    for i in range(1, len(seqs), 2):
        days.append([seqs[i-1], seqs[i]])
        profits = profits + capacity*(arr[seqs[i]] - arr [seqs[i-1]])
    return profits, days

In [8]:
# finaly, check if the function to find trading strategy and maximized profit works

mxProf(df.Close.values, 200)

(3065.2000000000025,
 [[0, 1],
  [3, 4],
  [6, 8],
  [9, 11],
  [13, 15],
  [16, 18],
  [19, 22],
  [24, 26],
  [28, 30],
  [34, 37],
  [39, 40],
  [42, 44],
  [46, 48],
  [51, 54],
  [56, 60],
  [63, 65],
  [67, 71],
  [74, 77],
  [79, 82],
  [85, 87],
  [91, 93],
  [95, 96],
  [97, 99],
  [100, 103],
  [105, 106],
  [108, 109],
  [110, 112],
  [117, 122],
  [125, 128],
  [130, 132],
  [134, 139],
  [140, 142],
  [144, 146],
  [149, 152],
  [153, 157],
  [159, 160],
  [162, 164],
  [165, 166],
  [169, 171],
  [173, 175],
  [176, 180],
  [181, 182],
  [184, 185],
  [188, 190],
  [191, 192],
  [193, 195],
  [197, 199],
  [202, 203],
  [204, 206],
  [207, 208],
  [210, 214],
  [217, 219],
  [221, 224],
  [225, 227],
  [230, 231],
  [234, 235],
  [238, 240],
  [241, 242],
  [244, 249],
  [251, 263],
  [266, 268],
  [270, 271],
  [273, 274],
  [276, 277],
  [279, 281],
  [283, 284],
  [287, 289],
  [292, 295],
  [297, 299],
  [300, 302],
  [304, 308],
  [312, 313],
  [315, 318],
  [323, 32